# Friedmann equations

This notebook demonstrates a few capabilities of SageMath in computations regarding cosmological spacetimes with Friedmann-Lemaître-Robertson-Walker (FLRW) metrics.
The corresponding tools have been developed within the  [SageManifolds](https://sagemanifolds.obspm.fr) project (version 1.3, as included in SageMath 8.3).

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v1.3/SM_Friedmann_equations.ipynb) to download the notebook file (ipynb format). To run it, you must start SageMath within the Jupyter notebook, via the command `sage -n jupyter`

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [1]:
version()

'SageMath version 9.0, Release Date: 2020-01-01'

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [2]:
%display latex

We declare the spacetime M as a 4-dimensional Lorentzian manifold:

In [3]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


We introduce the standard FLRW coordinates, via the method `chart()`, the argument of which is a string expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [4]:
fr.<t,r,th,ph> = M.chart(r't r:[0,+oo) th:[0,pi]:\theta ph:[0,2*pi):\phi')
fr

Chart (M, (t, r, th, ph))

<p>Assuming that the speed of light c=1, let us define a few variables: Newton's constant $G$, the cosmological constant $\Lambda$, the spatial curvature constant $k$, the scale factor $a(t)$, the fluid proper density $\rho(t)$ and the fluid pressure $p(t)$:</p>

In [5]:
var('G, Lambda, k', domain='real')
a = M.scalar_field(function('a')(t), name='a')
rho = M.scalar_field(function('rho')(t), name='rho')
p = M.scalar_field(function('p')(t), name='p')

The HU metric is defined by its components in the manifold's default frame, i.e. the frame associated with the FLRW coordinates:

In [6]:
# HU Metric
var("R0", "G0")
k=R0/t
G=G0*R0/t
a=t/R0
g = M.metric()
g[0,0] = -1
g[1,1] = a*a
g[2,2] = a*a*r^2
g[3,3] = a*a*(r*sin(th))^2
g.display()

g = -dt*dt + t^2/R0^2 dr*dr + r^2*t^2/R0^2 dth*dth + r^2*t^2*sin(th)^2/R0^2 dph*dph

<p>A matrix view of the metric components:</p>

In [7]:
g[:]

[                    -1                      0                      0                      0]
[                     0               t^2/R0^2                      0                      0]
[                     0                      0           r^2*t^2/R0^2                      0]
[                     0                      0                      0 r^2*t^2*sin(th)^2/R0^2]

<p>The Levi-Civita connection associated with the metric is computed:</p>

In [8]:
nabla = g.connection()
g.christoffel_symbols_display()

Gam^t_r,r = t/R0^2 
Gam^t_th,th = r^2*t/R0^2 
Gam^t_ph,ph = r^2*t*sin(th)^2/R0^2 
Gam^r_t,r = 1/t 
Gam^r_th,th = -r 
Gam^r_ph,ph = -r*sin(th)^2 
Gam^th_t,th = 1/t 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_t,ph = 1/t 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

In [9]:
g.christoffel_symbols_display?

Ricci tensor:

In [10]:
Ricci = nabla.ricci()
Ricci.display()

Ric(g) = 2/R0^2 dr*dr + 2*r^2/R0^2 dth*dth + 2*r^2*sin(th)^2/R0^2 dph*dph

In [11]:
Ricci.display_comp()

Ric(g)_r,r = 2/R0^2 
Ric(g)_th,th = 2*r^2/R0^2 
Ric(g)_ph,ph = 2*r^2*sin(th)^2/R0^2

<p>Ricci scalar ($R^\mu_{\ \, \mu}$):</p>

In [12]:
Ricci_scalar = g.ricci_scalar()
Ricci_scalar.display()

r(g): M --> R
   (t, r, th, ph) |--> 6/t^2

<p>The fluid 4-velocity:</p>

In [13]:
u = M.vector_field('u')
u[0] = 1
u.display()

u = d/dt

In [14]:
g(u,u).expr()

-1

`u.dot(u)` is equivalent to `g(u,u)`:

In [15]:
u.dot(u).expr()

-1

<p>Perfect fluid energy-momentum tensor $T$:</p>

In [16]:
u_form = u.down(g) # the 1-form associated to u by metric duality
T = (rho+p)*(u_form*u_form) + p*g
T.set_name('T')
print(T)
T.display()

Field of symmetric bilinear forms T on the 4-dimensional Lorentzian manifold M


T = rho(t) dt*dt + t^2*p(t)/R0^2 dr*dr + r^2*t^2*p(t)/R0^2 dth*dth + r^2*t^2*p(t)*sin(th)^2/R0^2 dph*dph

<p>The trace of $T$ (we use index notation to denote the double contraction $g^{ab} T_{ab}$):</p>

In [17]:
Ttrace = g.inverse()['^ab']*T['_ab']
Ttrace.display()

M --> R
(t, r, th, ph) |--> 3*p(t) - rho(t)

In [18]:
#HU make Lambda=0
Lambda=0

<p>Einstein equation: $R_{\mu \nu} - {1 \over 2} R g_{\mu \nu} + \Lambda g_{\mu \nu} = {8 \pi a^2 G} T_{\mu \nu}$</p>

In [19]:
E1 = Ricci - Ricci_scalar/2*g + Lambda*g - a**2*(8*pi*G)*T
print("First Friedmann equation:\n")
E1[0,0].expr().expand() == 0

First Friedmann equation:



-8*pi*G0*t*rho(t)/R0 + 3/t^2 == 0

<p>Trace-reversed version of the Einstein equation: $R_{\mu \nu} - \Lambda g_{\mu \nu} = {8 \pi a^2 G} \left(T_{\mu \nu} - {1 \over 2}T\,g_{\mu \nu}\right)$</p>

In [20]:
E2 = Ricci - Lambda*g - a**2*(8*pi*G)*(T - Ttrace/2*g)
print("Second Friedmann equation:\n")
E2[0,0].expr().expand() == 0

Second Friedmann equation:



-12*pi*G0*t*p(t)/R0 - 4*pi*G0*t*rho(t)/R0 == 0

# Hence
# $$3p(t) + \rho(t)=0$$
# $$ \rho(t) = \frac{3 R_0}{8\pi G_0  t^3} $$
# $$ p(t) = -\frac{ R_0}{8\pi G_0  t^3} $$